# was this competition ill conceived ?

Doing some aftermath a year after this competition, i think the competition was ill conceived.  The actual problem is that cars are tested during 100seconds on their emissions, and this test is evidently function of the properties of the cars.
The cars come during the production in a whole mix, and this variability induces another variability. The question to minimize the emission, could rather have been: what are the key parameters influencing the test, or what are the key causes of the delays and variability ?

What one is sure, that certain cars, and certain properties end up with a minimal 'test time'. The lowest test-time is probabily the realistic solution if the test would be executed robotically.
What is also sure, is applying the robust analysis, elmininates the influence of those outliers... and this method predicts the essential realistic testing time and not the measured time..
Hence since the competition benchmarked the testing time, you could try to forecast the testing time, what we did, but was fundamentally flawed. We should have tried to forecast the 'realistic time'


# import

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score



train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
print(train.shape,test.shape)
ytrain=train['y']

X0=pd.DataFrame(train.groupby('X0').min()['y'])
train2=pd.merge(train,X0,how='outer',left_on='X0',right_index=True,sort=False)
train2['diff']=train2['y_x']-train2['y_y']
difftrain=train2['diff'].sort_index()


train['trte']=0
test['trte']=1
train=train.append(test)

print(train.head(),train.shape)

for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))
   

#Xtrain=Xtrain.sort_values(['y','ID'])


Xtrain=train.drop(['y','ID'],axis=1)
print(train.head())

# the difference between Min of each X0 group

what explains the difference ?

In [ ]:
def cohen_effect_size(X, y):
    """Calculates the Cohen effect size of each feature.
    
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vector, where n_samples in the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target vector relative to X
        Returns
        -------
        cohen_effect_size : array, shape = [n_features,]
            The set of Cohen effect values.
        Notes
        -----
        Based on https://github.com/AllenDowney/CompStats/blob/master/effect_size.ipynb
    """
    print(X.shape,y.shape,y.mean())
    medi=y.mean()
    group1, group2 = X[y<medi], X[y>=medi]
    diff = group1.mean() - group2.mean()
    var1, var2 = group1.var(), group2.var()
    n1, n2 = group1.shape[0], group2.shape[0]
    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    d = diff / np.sqrt(pooled_var)
    return d

# Most siginificant features explaining the test time y

higly correlated X314,X127,X261

In [ ]:


excluded_feats = [] #['SK_ID_CURR']

features = [f_ for f_ in Xtrain.columns if f_ not in excluded_feats]
print('Number of features %d' % len(features),Xtrain.shape,ytrain.shape)
#effect_sizes = cohen_effect_size(Xtrain[:len(ytrain)], ytrain)
effect_sizes = cohen_effect_size(Xtrain[:len(ytrain)],ytrain)
effect_sizes.reindex(effect_sizes.abs().sort_values(ascending=False).nlargest(50).index)[::-1].plot.barh(figsize=(6, 10));
print('Features with the 30 largest effect sizes')
significant_features = [f for f in features if np.abs(effect_sizes.loc[f]) > 0.1]
print('Significant features %d: %s' % (len(significant_features), significant_features))

# Most siginificant features explaining the test time difference

explaining with relative low correlation 0.2
One could conclude there is no very good explanation for that variability X47, X267 are probably the best explanation

In [ ]:


excluded_feats = [] #['SK_ID_CURR']

features = [f_ for f_ in Xtrain.columns if f_ not in excluded_feats]
print('Number of features %d' % len(features),Xtrain.shape,ytrain.shape)
#effect_sizes = cohen_effect_size(Xtrain[:len(ytrain)], ytrain)
effect_sizes = cohen_effect_size(Xtrain[:len(ytrain)],pd.DataFrame(difftrain).reset_index().set_index('index')['diff'])
effect_sizes.reindex(effect_sizes.abs().sort_values(ascending=False).nlargest(50).index)[::-1].plot.barh(figsize=(6, 10));
print('Features with the 30 largest effect sizes')
significant_features2 = [f for f in features if np.abs(effect_sizes.loc[f]) > 0.1]
print('Significant features %d: %s' % (len(significant_features2), significant_features2))

# Plotting most relevant features 3D

plotting time spend on test, verss X0 , versus most relevant features

In [ ]:
from time import time

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

from sklearn import manifold
from sklearn.utils import check_random_state

# Next line to silence pyflakes.
Axes3D

colors = np.round(np.log(train2['y_x']))#np.round(train['y'])


x, y, z =  train['y'],train['X314'],Xtrain['X0']
# Plot our dataset.
fig = plt.figure(figsize=(45, 22))
ax = fig.add_subplot(251, projection='3d')
ax.scatter(x, y, z, c=colors, cmap=plt.cm.rainbow)
ax.view_init(25, -25)

x, y, z = train['y'],train['X261'],Xtrain['X0']
# Plot our dataset.
fig = plt.figure(figsize=(45, 22))
ax = fig.add_subplot(252, projection='3d')
ax.scatter(x, y, z, c=colors, cmap=plt.cm.rainbow)
ax.view_init(25, -25)

x, y, z = train['y'],train['X127'],Xtrain['X0']
# Plot our dataset.
fig = plt.figure(figsize=(45, 22))
ax = fig.add_subplot(253, projection='3d')
ax.scatter(x, y, z, c=colors, cmap=plt.cm.rainbow)
ax.view_init(25, -25)

x, y, z = train['y'],train['X225'],Xtrain['X150']
# Plot our dataset.
fig = plt.figure(figsize=(45, 22))
ax = fig.add_subplot(254, projection='3d')
ax.scatter(x, y, z, c=colors, cmap=plt.cm.rainbow)
ax.view_init(25, -25)


plt.show()

# splitting data
in a group effect A_ matrix, is forecasting the basic matrix without any variability.

and an error matrix e_ is the leftover part.

# first find important features

In [ ]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(Xtrain[significant_features])
rejected_variables = profile.get_rejected_variables(threshold=0.95)
selected_features = list(set(significant_features) - set(rejected_variables))
print(selected_features)

In [ ]:
import numpy as np
from numpy.linalg import norm, svd
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
#svd = TruncatedSVD(n_components=int(disc.shape[1]-1), n_iter=7, random_state=42)


def robustSVD(X,n_comp,lmbda=.01, tol=1e-3, maxiter=100, verbose=True):
    svd = TruncatedSVD(n_components=n_comp, n_iter=10, random_state=42)
    """
    Inexact Augmented Lagrange Multiplier
    """
    Y = X
    norm_two = norm(Y.ravel(), 2)
    norm_inf = norm(Y.ravel(), np.inf) / lmbda
    dual_norm = np.max([norm_two, norm_inf])
    Y = Y / dual_norm
    A = np.zeros(Y.shape)
    E = np.zeros(Y.shape)
    dnorm = norm(X, 'fro')
    mu = 1.25 / norm_two
    rho = 1.5
    sv = 10.
    n = Y.shape[0]
    itr = 0
    while True:
        Eraw = X - A + (1 / mu) * Y
        Eupdate = np.maximum(Eraw - lmbda / mu, 0) + np.minimum(Eraw + lmbda / mu, 0)
        #U, S, V = svd(X - Eupdate + (1 / mu) * Y, full_matrices=False)
        X_n=svd.fit_transform(X - Eupdate + (1 / mu) * Y)
        S=svd.singular_values_
        if itr ==0:
            pd.DataFrame(svd.explained_variance_ratio_*100).plot()
        svp = (S > 1 / mu).shape[0]
        print(S.sum())
        if svp < sv:
            sv = np.min([svp + 1, n])
        else:
            sv = np.min([svp + round(.05 * n), n])
        #Aupdate = np.dot(np.dot(U[:, :svp], np.diag(S[:svp] - 1 / mu)), V[:svp, :])
        Aupdate = svd.inverse_transform(X_n)
        A = Aupdate
        E = Eupdate
        Z = X - A - E
        Y = Y + mu * Z
        mu = np.min([mu * rho, mu * 1e7])
        itr += 1
        
        if ((norm(Z, 'fro') / dnorm) < tol) or (itr >= maxiter):
            u=svd.components_
            pd.DataFrame(svd.explained_variance_ratio_*100).plot()
            pd.DataFrame(A[:33,:20]).plot()
            pd.DataFrame(E[:33,:20]).plot()
            break
    if verbose:
        print("Finished at iteration %d" % (itr))  
    return A, E, u, S

A_,e_,u_,s_=robustSVD(Xtrain[selected_features].values,1)

# finding significant features again from the A_ matrix
you can see there aren't anymore highly correlated features, are they all hoover around +-0.4

In [ ]:
excluded_feats = [] #['SK_ID_CURR']

features = [f_ for f_ in Xtrain[selected_features].columns if f_ not in excluded_feats]
print('Number of features %d' % len(features),Xtrain.shape,ytrain.shape)
#effect_sizes = cohen_effect_size(Xtrain[:len(ytrain)], ytrain)
effect_sizes = cohen_effect_size(pd.DataFrame(A_,columns=Xtrain[selected_features].columns)[:len(ytrain)],ytrain)
effect_sizes.reindex(effect_sizes.abs().sort_values(ascending=False).nlargest(30).index)[::-1].plot.barh(figsize=(6, 10));
print('Features with the 30 largest effect sizes')
significant_features2 = [f for f in features if np.abs(effect_sizes.loc[f]) > 0.05]
print('Significant features %d: %s' % (len(significant_features2), significant_features2))

# selecting the essential features

In [ ]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(pd.DataFrame(A_,columns=Xtrain[selected_features].columns))
rejected_variables = profile.get_rejected_variables(threshold=0.95)
selected_features3 = list(set(significant_features2) - set(rejected_variables))
print(selected_features3)

# doing gradient boost with selected features
as you can see the influence of the outlier disappeared

In [ ]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

start=len(train)-len(test)
t_=pd.DataFrame(A_,columns=Xtrain[selected_features].columns)
#X = similarities[:start]
#Y=train[:start]['y'].round()
print(t_.shape)
x_train, x_valid, y_train, y_valid = train_test_split((t_[:start])[selected_features3], ytrain[:start], test_size=0.15, random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix((t_[start:])[selected_features3] )

params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.17
params['gamma'] = 0.125
params['max_depth'] = 50
params['min_child_weight'] = 1 
params['num_round'] = 7

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
#feval=xgb_r2_score, maximize=True,

clf = xgb.train(params, d_train, 2000, watchlist, early_stopping_rounds=10,  verbose_eval=10)

pre = pd.DataFrame()
pre['y']= y_train
pre['pre']=clf.predict(d_train)
pre.plot.scatter('y','pre')
sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['y'] = clf.predict(d_test)
sub.to_csv('xgb_a.csv', index=False)

In [ ]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

start=len(train)-len(test)
t_=pd.DataFrame(e_,columns=Xtrain[selected_features].columns)
#X = similarities[:start]
#Y=train[:start]['y'].round()
print(t_.shape)
x_train, x_valid, y_train, y_valid = train_test_split((t_[:start])[selected_features3], ytrain[:start], test_size=0.15, random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix((t_[start:])[selected_features3] )

params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.17
params['gamma'] = 0.125
params['max_depth'] = 50
params['min_child_weight'] = 1 
params['num_round'] = 7

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
#feval=xgb_r2_score, maximize=True,

clf = xgb.train(params, d_train, 2000, watchlist, early_stopping_rounds=10,  verbose_eval=10)

pre = pd.DataFrame()
pre['y']= y_train
pre['pre']=clf.predict(d_train)
pre.plot.scatter('y','pre')
sub = pd.DataFrame()
sub['ID'] = test['ID']
sub['y'] = clf.predict(d_test)
sub.to_csv('xgb_e.csv', index=False)